# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
%%time
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';',
                     parse_dates=True, keep_date_col=True)

In [ ]:
%%time
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';', 
                        parse_dates=True, keep_date_col=True,)

In [ ]:
azdias.head()

In [ ]:
azdias.shape

In [ ]:
customers.shape

In [ ]:
azdias.info()

In [ ]:
customers.info()

As it can be seen above, reading the csv files takes more than 12 mins. I decided to decrease the memory usage for the files to make the process of data analysis more efficient.

I took the memory usage function from the following [repository](https://github.com/harshdarji23/Arvato-Identifying-the-potential-customers/blob/master/Jupyter%20Notebook/Arvato%20Project%20Workbook%20-Final.ipynb), which was created from this [source code from Kaggle](https://www.kaggle.com/gemartin/load-data-reduce-memory-usage).

In [ ]:
def reduce_mem_usage(df):
    """ This function iterates through all the columns of a dataframe and modifies the data type
        to reduce memory usage. 
    Parameters:
    df (pandas DataFrame): a dataframe, which needs to be reduced
    
    Returns:
    df (pandas DataFrame): a dataframe with a reduced memory usage
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
%%time
azdias_red_mem = reduce_mem_usage(azdias)

In [ ]:
%%time
customers_red_mem = reduce_mem_usage(customers)

Let us save the optimized files

In [ ]:
azdias_red_mem.to_csv('Udacity_AZDIAS_reduced_memory_052018.csv', index=False)

In [ ]:
customers_red_mem.to_csv('Udacity_CUSTOMERS_reduced_memory_052018.csv', index=False)

In [2]:
%%time
# Read files with reduced memory
azdias = pd.read_csv('Udacity_AZDIAS_reduced_memory_052018.csv')

<string>:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 42.4 s, sys: 13.7 s, total: 56.2 s
Wall time: 11min 2s


In [3]:
%%time
customers = pd.read_csv('Udacity_CUSTOMERS_reduced_memory_052018.csv')

<string>:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 14.8 s, sys: 3.08 s, total: 17.9 s
Wall time: 2min 46s


In [4]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [5]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [6]:
azdias.shape

(891221, 366)

In [7]:
customers.shape

(191652, 369)

In [8]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [9]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


### Fixing the warning messages

In [10]:
# Read information on the attributes 
attributes_vals = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header=1)
attributes_info = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header=1)

In [23]:
attributes_vals.head(10)

,Attribute,Description,Value,Meaning
NaN,AGER_TYP,best-ager typology,-1,unknown
NaN,NaN,NaN,0,no classification possible
NaN,NaN,NaN,1,passive elderly
NaN,NaN,NaN,2,cultural elderly
NaN,NaN,NaN,3,experience-driven elderly
NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
NaN,NaN,NaN,1,< 30 years
NaN,NaN,NaN,2,30 - 45 years
NaN,NaN,NaN,3,46 - 60 years
NaN,NaN,NaN,4,> 60 years


In [26]:
# Let us fill in the null values in the attribute column
attributes_vals["Attribute"] = attributes_vals["Attribute"].ffill()
attributes_vals.head(15)

,Attribute,Description,Value,Meaning
NaN,AGER_TYP,best-ager typology,-1,unknown
NaN,AGER_TYP,NaN,0,no classification possible
NaN,AGER_TYP,NaN,1,passive elderly
NaN,AGER_TYP,NaN,2,cultural elderly
NaN,AGER_TYP,NaN,3,experience-driven elderly
NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
NaN,ALTERSKATEGORIE_GROB,NaN,1,< 30 years
NaN,ALTERSKATEGORIE_GROB,NaN,2,30 - 45 years
NaN,ALTERSKATEGORIE_GROB,NaN,3,46 - 60 years
NaN,ALTERSKATEGORIE_GROB,NaN,4,> 60 years


In [24]:
attributes_info.head(10)

,Information level,Attribute,Description,Additional notes
NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
NaN,NaN,ANREDE_KZ,gender,NaN
NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
NaN,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
NaN,NaN,FINANZ_SPARER,financial typology: money saver,NaN
NaN,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN
NaN,NaN,FINANZ_ANLEGER,financial typology: investor,NaN
NaN,NaN,FINANZ_UNAUFFAELLIGER,financial typology: unremarkable,NaN
NaN,NaN,FINANZ_HAUSBAUER,financial typology: main focus is the own house,NaN


In [13]:
attributes_vals.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2258 entries, nan to nan
Data columns (total 4 columns):
Attribute      314 non-null object
Description    351 non-null object
Value          2258 non-null object
Meaning        2247 non-null object
dtypes: object(4)
memory usage: 88.2+ KB


In [14]:
attributes_info.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 313 entries, nan to nan
Data columns (total 4 columns):
Information level    10 non-null object
Attribute            313 non-null object
Description          313 non-null object
Additional notes     20 non-null object
dtypes: object(4)
memory usage: 12.2+ KB


### 0.1 Data Cleaning

### Verify Integrity

In [15]:
# Print the shapes for the dataframes
print("Azdias shape:", azdias.shape)
print("Customers shape:", customers.shape)

Azdias shape: (891221, 366)
Customers shape: (191652, 369)


Let's check the number of null values

In [16]:
azdias.isnull().sum().sum()

33492923

In [17]:
customers.isnull().sum().sum()

13864522

In [18]:
# Print the percentage of missing values for each dataset
print("Percentage of missing values for Azdias:", 
      azdias.isnull().sum().sum()*100/(azdias.shape[0]*azdias.shape[1]))
print("Percentage of missing values for Customers:",
      customers.isnull().sum().sum()*100/(customers.shape[0]*customers.shape[1]))

Percentage of missing values for Azdias: 10.2680164156
Percentage of missing values for Customers: 19.6049247346


Let's check the number of duplicated values

In [19]:
azdias.duplicated().sum().sum()

0

In [20]:
customers.duplicated().sum().sum()

0

There are no duplicated rows in two datasets.

It seems that a dataframe with customers'data has 3 extra columns.

In [21]:
# Check the extra columns
set_diff = set(list(customers.columns)) - set(list(azdias.columns))
extra_cols = list(set_diff)
print("Extra columns in Customers dataset:", extra_cols)

Extra columns in Customers dataset: ['CUSTOMER_GROUP', 'PRODUCT_GROUP', 'ONLINE_PURCHASE']


In [22]:
customers[extra_cols].head()

,CUSTOMER_GROUP,PRODUCT_GROUP,ONLINE_PURCHASE
0,MULTI_BUYER,COSMETIC_AND_FOOD,0
1,SINGLE_BUYER,FOOD,0
2,MULTI_BUYER,COSMETIC_AND_FOOD,0
3,MULTI_BUYER,COSMETIC,0
4,MULTI_BUYER,FOOD,0


### Fixing warning messages

In [29]:
# Let's have a look at the columns with warnings
azdias.columns[18:20]

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')

Let us check those values in a dataframe with attribute values

In [31]:
attributes_vals[attributes_vals["Attribute"] == "CAMEO_DEUG_2015"]

,Attribute,Description,Value,Meaning
NaN,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
NaN,CAMEO_DEUG_2015,NaN,1,upper class
NaN,CAMEO_DEUG_2015,NaN,2,upper middleclass
NaN,CAMEO_DEUG_2015,NaN,3,established middleclasse
NaN,CAMEO_DEUG_2015,NaN,4,consumption-oriented middleclass
NaN,CAMEO_DEUG_2015,NaN,5,active middleclass
NaN,CAMEO_DEUG_2015,NaN,6,low-consumption middleclass
NaN,CAMEO_DEUG_2015,NaN,7,lower middleclass
NaN,CAMEO_DEUG_2015,NaN,8,working class
NaN,CAMEO_DEUG_2015,NaN,9,urban working class


In [33]:
attributes_vals[attributes_vals["Attribute"] == "CAMEO_DEUINTL_2015"]

,Attribute,Description,Value,Meaning
NaN,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
NaN,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,11,Wealthy Households-Pre-Family Couples & Singles
NaN,CAMEO_DEUINTL_2015,NaN,12,Wealthy Households-Young Couples With Children
NaN,CAMEO_DEUINTL_2015,NaN,13,Wealthy Households-Families With School Age Ch...
NaN,CAMEO_DEUINTL_2015,NaN,14,Wealthy Households-Older Families & Mature Co...
NaN,CAMEO_DEUINTL_2015,NaN,15,Wealthy Households-Elders In Retirement
NaN,CAMEO_DEUINTL_2015,NaN,21,Prosperous Households-Pre-Family Couples & Sin...
NaN,CAMEO_DEUINTL_2015,NaN,22,Prosperous Households-Young Couples With Children
NaN,CAMEO_DEUINTL_2015,NaN,23,Prosperous Households-Families With School Age...
NaN,CAMEO_DEUINTL_2015,NaN,24,Prosperous Households-Older Families & Mature ...


Based on the attribute values dataframe, it's clear that the values should be integer.

In [36]:
warning_columns = list(azdias.columns[18:20])

In [37]:
azdias[warning_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Data columns (total 2 columns):
CAMEO_DEUG_2015    792242 non-null object
CAMEO_INTL_2015    792242 non-null object
dtypes: object(2)
memory usage: 13.6+ MB


The warning columns have object type instead of integer.

In [38]:
# Let's have a closer look at the values in the warning columns
azdias[warning_columns[0]].unique()

array([nan, 8.0, 4.0, 2.0, 6.0, 1.0, 9.0, 5.0, 7.0, 3.0, '4', '3', '7',
       '2', '8', '9', '6', '5', '1', 'X'], dtype=object)

In [39]:
azdias[warning_columns[1]].unique()

array([nan, 51.0, 24.0, 12.0, 43.0, 54.0, 22.0, 14.0, 13.0, 15.0, 33.0,
       41.0, 34.0, 55.0, 25.0, 23.0, 31.0, 52.0, 35.0, 45.0, 44.0, 32.0,
       '22', '24', '41', '12', '54', '51', '44', '35', '23', '25', '14',
       '34', '52', '55', '31', '32', '15', '13', '43', '33', '45', 'XX'], dtype=object)

The solumns contain nan, 'XX' and 'X' values, which are not in the attribute list of values. Therefore, let's replace thoose values by -1, which means unknown information.

In [ ]:
list(azdias.columns)

In [ ]:
list(customers.columns)

In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')